In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding = "ISO-8859-1")

In [3]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])

In [4]:
vendas['DATE_BLOCK'] = LabelEncoder().fit_transform(vendas.ANO.apply(str) + vendas.MES.apply(lambda x: str(x).zfill(5)))

### Fazer as vendas mensais 

In [5]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'DATE_BLOCK'])

In [6]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [7]:
last_year = np.array([])
for i in np.unique(df.DATE_BLOCK):
    media = list(filter(lambda x: x>=0, [j for j in range(i-1, i-13, -1)]))
    df2 = df[df.DATE_BLOCK.isin(media)]
    df2 = df2.groupby(['DESCRICAO','NOME', 'TAMANHO']).sum().reset_index()
    a = df[df.DATE_BLOCK == i].merge(df2[['NOME', 'DESCRICAO', 'TAMANHO', 'QUANTIDADE']], on=['NOME', 'DESCRICAO', 'TAMANHO'], how='left')
    last_year = np.append(last_year, (a.QUANTIDADE_y).fillna(-1))

In [8]:
df['last_year'] = last_year

In [9]:
treino = df[df.ANO < 2018]
validacao = df[df.ANO >= 2018]

In [11]:
treino.tail()

,ANO,MES,DESCRICAO,NOME,TAMANHO,DATE_BLOCK,QUANTIDADE,last_year
4391,2017.0,12.0,SHORT SAIA,ITAIM,00,23,2.0,4.0
4392,2017.0,12.0,SHORT SAIA,VERBO DIVINO,02,23,1.0,26.0
4393,2017.0,12.0,SHORT SAIA,VERBO DIVINO,06,23,3.0,91.0
4394,2017.0,12.0,SHORT SAIA,VERBO DIVINO,08,23,2.0,43.0
4395,2017.0,12.0,SHORT SAIA,VERBO DIVINO,10,23,1.0,31.0


#### Data

In [20]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

In [13]:
le_DESC = LabelEncoder()
treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)

In [14]:
le_TAM = LabelEncoder()
treino['TAMANHO'] = le_TAM.fit_transform(treino.TAMANHO)
validacao['TAMANHO'] = le_TAM.transform(validacao.TAMANHO)

In [15]:
le_NOME = LabelEncoder()
treino['NOME'] = le_NOME.fit_transform(treino.NOME)
validacao['NOME'] = le_NOME.transform(validacao.NOME)

In [16]:
y_train = treino['QUANTIDADE']
y_val = validacao['QUANTIDADE']

X_train = treino.drop(['QUANTIDADE', 'DATE_BLOCK'], 1)
X_val = validacao.drop(['QUANTIDADE', 'DATE_BLOCK'], 1)

In [17]:
one = OneHotEncoder(categorical_features=[1,2, 3, 4])
X_train_ = one.fit_transform(X_train).toarray()
X_val_ = one.transform(X_val).toarray()

In [18]:
# treino[treino.ANO == 2017]

## Treino

In [21]:
sc = StandardScaler()
X_train_ = sc.fit_transform(X_train)
X_val_ = sc.transform(X_val)

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

Using TensorFlow backend.


In [23]:
def ret_model():
    model = Sequential()
    model.add(Dense(59, input_dim=X_train_.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, input_dim=X_train_.shape[1], kernel_initializer='normal', activation='elu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [24]:
calbacks = [ModelCheckpoint(filepath='./model', monitor='val_loss', save_best_only=True, verbose=1), EarlyStopping(patience=20)]
model = ret_model()

In [25]:
# model.fit(X_train_, y_train, validation_data=(X_val_, y_val), callbacks=calbacks,  epochs=1000, verbose=1)
# model.load_weights('./model')

In [26]:
# y_lin = model.predict(X_val_)
# np.array(y_val[100:120]), y_lin[100:120]

In [27]:
rf = RandomForestRegressor(criterion='mse',n_estimators=125, min_samples_leaf=2, random_state=3, n_jobs=-1)

In [28]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=125, n_jobs=-1,
           oob_score=False, random_state=3, verbose=0, warm_start=False)

In [29]:
ext = ExtraTreesRegressor(criterion='mse',n_estimators=125, max_depth=14, min_samples_leaf=5, random_state=3, n_jobs=-1)
ext = ExtraTreesRegressor(criterion='mse',n_estimators=200, max_depth=15, random_state=3, n_jobs=-1)

In [30]:
ext.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=15,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
          oob_score=False, random_state=3, verbose=0, warm_start=False)

In [31]:
y_rf = rf.predict(X_val)

In [32]:
y_ext = ext.predict(X_val)

In [33]:
dict(zip(X_train.columns, rf.feature_importances_))

{'ANO': 0.041117012566097816,
 'MES': 0.3224793475543148,
 'DESCRICAO': 0.1344313463432912,
 'NOME': 0.08659593044035953,
 'TAMANHO': 0.11608902876893579,
 'last_year': 0.29928733432700066}

In [34]:
dict(zip(X_train.columns, ext.feature_importances_))

{'ANO': 0.04168956371722239,
 'MES': 0.33467144628405876,
 'DESCRICAO': 0.16221869194969749,
 'NOME': 0.07247567813765399,
 'TAMANHO': 0.10630512123409158,
 'last_year': 0.28263949867727584}

### Validação

In [35]:
from sklearn.metrics import mean_squared_error

In [36]:
mean_squared_error(y_val, y_rf)

37.73674131143746

In [37]:
mean_squared_error(y_val, y_ext)

47.65846730626679

In [38]:
mean_squared_error(y_val, list(map(up, y_lin)))

NameError: name 'up' is not defined

In [39]:
mean_squared_error(validacao.QUANTIDADE.values, (y_rf + y_ext)/2)

40.15746631349165

### Benchmark

In [40]:
def bench(u, a_):
    try:
        return a_[u]
    except:
        return 1

In [41]:
treino['unique_id'] = treino.DESCRICAO.astype(str).apply(lambda s : s.zfill(2)) + treino.NOME.astype(str).apply(lambda s : s.zfill(2)) + treino.TAMANHO.astype(str).apply(lambda s : s.zfill(2)) + treino.MES.astype(str).apply(lambda s : s.zfill(2))
b1 = treino.groupby(['MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'unique_id']).mean().reset_index()
b2 = treino[treino.ANO == 2017].groupby(['MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'unique_id']).mean().reset_index()
b_1 = dict(zip(b1.unique_id, b1.QUANTIDADE))
b_2 = dict(zip(b2.unique_id, b2.QUANTIDADE))

In [42]:
validacao['unique_id'] = validacao.DESCRICAO.astype(str).apply(lambda s : s.zfill(2)) + validacao.NOME.astype(str).apply(lambda s : s.zfill(2)) + validacao.TAMANHO.astype(str).apply(lambda s : s.zfill(2)) + validacao.MES.astype(str).apply(lambda s : s.zfill(2))

In [43]:
df['unana'] = df.groupby(['DESCRICAO', 'NOME', 'TAMANHO'])['QUANTIDADE'].shift(12).fillna(-1) #+ df.groupby(['DESCRICAO', 'NOME'])['QUANTIDADE'].shift(24).fillna(-1)) / 2

In [44]:
df[df.ANO == 2018]

,ANO,MES,DESCRICAO,NOME,TAMANHO,DATE_BLOCK,QUANTIDADE,last_year,unana
4399,2018.0,1.0,BERMUDA CICLISTA COTTON,ALDEIA DA SERRA,02,24,1.0,2.0,-1.0
4402,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,02,24,3.0,5.0,-1.0
4396,2018.0,1.0,BERMUDA CICLISTA COTTON,ACLIMAÇÃO,04,24,8.0,9.0,-1.0
4400,2018.0,1.0,BERMUDA CICLISTA COTTON,ALDEIA DA SERRA,04,24,1.0,6.0,-1.0
4403,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,04,24,13.0,33.0,-1.0
4397,2018.0,1.0,BERMUDA CICLISTA COTTON,ACLIMAÇÃO,06,24,12.0,14.0,-1.0
4401,2018.0,1.0,BERMUDA CICLISTA COTTON,ALDEIA DA SERRA,06,24,2.0,1.0,-1.0
4404,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,06,24,20.0,27.0,-1.0
4398,2018.0,1.0,BERMUDA CICLISTA COTTON,ACLIMAÇÃO,08,24,8.0,4.0,-1.0
4405,2018.0,1.0,BERMUDA CICLISTA COTTON,VERBO DIVINO,08,24,13.0,34.0,6.0


In [45]:
media_sazonal = validacao.unique_id.apply(lambda u: bench(u, b_1))

In [46]:
ultimo_ano = validacao.unique_id.apply(lambda u: bench(u, b_2))

In [47]:
mean_squared_error(validacao.QUANTIDADE.values, df[df.ANO == 2018]['unana'])

281.4020304568528

In [48]:
mean_squared_error(validacao.QUANTIDADE.values, ultimo_ano)

64.12791878172588

In [49]:
pd.DataFrame({
    'X Real' : validacao.QUANTIDADE,
    'Y RF' : np.round((y_rf + y_ext)/2),
    'Z Media sazonal': media_sazonal
}).tail()

,X Real,Y RF,Z Media sazonal
5376,1.0,3.0,1.0
5377,1.0,3.0,3.5
5378,3.0,3.0,2.0
5379,5.0,3.0,1.0
5380,2.0,3.0,1.0


In [50]:
validacao = df[df.ANO >= 2018]

In [51]:
validacao['Y RF'] = (y_rf + y_ext)/2

In [52]:
validacao['Z Media sazonal'] = media_sazonal